In [4]:
import random
POPULATION_SIZE = 10
number_of_parents = 10
CROSSOVER_RATE = 0.7
MUTATION_RATE = 0.1
EPOCHS = 500

class Classroom:
    def __init__(self, name, capacity, room_type):
        self.name = name
        self.capacity = capacity
        self.room_type = room_type
    def __str__(self):
        return f"Classroom {self.name} (Capacity: {self.capacity}, Type: {self.room_type})"

class Professor:
    def __init__(self, name, preferred_days=[]):
        self.name = name
        self.preferred_days = preferred_days

class Assistant:
    def __init__(self, name):
        self.name = name

class Lesson:
    def __init__(self, code, teacher, name, is_online, class_type, preferred_days=[], timeslot=None, duration=1, room_preference=[], capacity=None):
        self.code = code
        self.teacher = teacher
        self.name = name
        self.is_online = is_online
        self.class_type = class_type
        self.preferred_days = preferred_days
        self.timeslot = timeslot
        self.duration = duration
        self.room_preference = room_preference
        self.capacity = capacity

    def __str__(self):
        return self.code

class ScheduledLesson:
    def __init__(self, lesson, classroom, day, start_timeslot):
        self.lesson = lesson
        self.classroom = classroom
        self.day = day
        self.start_timeslot = start_timeslot

    def __str__(self):
        return f"[{self.lesson.code}, {self.lesson.teacher.name}, {self.classroom.name}, {self.day}, {self.start_timeslot}]"

class Schedule:
    def __init__(self, scheduled_lessons):
        self.scheduled_lessons = scheduled_lessons
        self.fitness = self.calculate_fitness()

    def __str__(self):
        return "\n".join(str(sl) for sl in self.scheduled_lessons)

    def calculate_fitness(self):
        return random.random()  # Placeholder for now


In [5]:
professors = [
    Professor("Dr. Smith", preferred_days=["Monday"]),
    Professor("Prof. Johnson", preferred_days=["Tuesday", "Thursday"]),
    Professor("Prof. Yıldız", preferred_days=["Tuesday", "Friday"])
]
classrooms = [
    Classroom("C1", capacity=30, room_type="Lecture"),
    Classroom("C2", capacity=25, room_type="Lecture"),
    Classroom("C3", capacity=20, room_type="Lecture"),
    Classroom("C4", capacity=20, room_type="Lecture"),
    Classroom("C5", capacity=15, room_type="Lecture"),
    Classroom("C6", capacity=15, room_type="Lab"),
    Classroom("C7", capacity=10, room_type="Lab")
]

lessons = [
  Lesson(
    code="INF101",
    teacher=professors[0],  # Dr. Smith
    name="Introduction to Computer Science",
    is_online=False,
    class_type="Lecture",
    duration=2,
    room_preference=classrooms[:2],  # Preferable classrooms C1, C2, C3
    capacity=30
  ),

  Lesson(
    code="INF104",
    teacher=professors[1],  # Prof. Johnson
    name="Data Structures and Algorithms",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference= [classrooms[0], classrooms[1], classrooms[2], classrooms[3]],
    capacity=25
  ),

  Lesson(
    code="INF205",
    teacher=professors[2],  # Prof. Yıldız
    name="Introduction to Artificial Intelligence",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference=classrooms[3:5],
    capacity=20
  ),

  Lesson(
    code="INF204",
    teacher=professors[0],  # Dr. Smith
    name="Database Management Systems",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference=classrooms[:3],
    capacity=30
  ),
  Lesson(
    code="INF304",
    teacher=professors[0],  # Dr. Smith
    name="Database Management Systems2",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference=classrooms[:3],
    capacity=30
  ),
  Lesson(
    code="INF404",
    teacher=professors[0],  # Dr. Smith
    name="Database Management Systems2",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference=classrooms[:3],
    capacity=30
  ),
  Lesson(
    code="INF504",
    teacher=professors[0],  # Dr. Smith
    name="Database Management Systems2",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference=classrooms[:3],
    capacity=30
  ),
  Lesson(
    code="INF604",
    teacher=professors[0],  # Dr. Smith
    name="Database Management Systems2",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference=classrooms[:3],
    capacity=30
  ),
  Lesson(
    code="INF704",
    teacher=professors[0],  # Dr. Smith
    name="Database Management Systems2",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference=classrooms[:3],
    capacity=30
  ),
  Lesson(
    code="INF303",
    teacher=professors[1],  # Prof. Johnson
    name="Software Engineering",
    is_online=False,
    class_type="Lab",
    duration=2,
    room_preference=classrooms[4:6],
    capacity=25
  )
]

days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
timeslots = ["9-10", "10-11", "11-12", "12-13", "13-14", "14-15", "15-16", "16-17", "17-18"]

In [6]:
def create_random_schedule():
    scheduled_lessons = []
    for lesson in lessons:

        day = random.choice(lesson.preferred_days if lesson.preferred_days else days_of_week)

        possible_start_slots = [i for i in range(len(timeslots) - lesson.duration + 1)]
        start_slot_index = random.choice(possible_start_slots)
        assigned_timeslots = timeslots[start_slot_index:start_slot_index + lesson.duration]

        room_candidates = [room for room in classrooms if room.name in lesson.room_preference]
        classroom = random.choice(room_candidates if room_candidates else classrooms)

        scheduled_lessons.append(ScheduledLesson(lesson, classroom, day, assigned_timeslots))

    return Schedule(scheduled_lessons)

def create_population():
    population = [create_random_schedule() for _ in range(POPULATION_SIZE)]
    return population

# Create pop
population = create_population()

for i, schedule in enumerate(population):
    print(f"Schedule {i+1}:")
    print(schedule)
    print("\n" + "="*20 + "\n")

Schedule 1:
[INF101, Dr. Smith, C4, Thursday, ['15-16', '16-17']]
[INF104, Prof. Johnson, C2, Friday, ['11-12', '12-13']]
[INF205, Prof. Yıldız, C3, Friday, ['13-14', '14-15']]
[INF204, Dr. Smith, C2, Friday, ['14-15', '15-16']]
[INF304, Dr. Smith, C7, Wednesday, ['9-10', '10-11']]
[INF404, Dr. Smith, C4, Thursday, ['11-12', '12-13']]
[INF504, Dr. Smith, C5, Tuesday, ['13-14', '14-15']]
[INF604, Dr. Smith, C5, Thursday, ['15-16', '16-17']]
[INF704, Dr. Smith, C5, Friday, ['13-14', '14-15']]
[INF303, Prof. Johnson, C4, Monday, ['12-13', '13-14']]


Schedule 2:
[INF101, Dr. Smith, C2, Thursday, ['13-14', '14-15']]
[INF104, Prof. Johnson, C7, Tuesday, ['12-13', '13-14']]
[INF205, Prof. Yıldız, C3, Wednesday, ['15-16', '16-17']]
[INF204, Dr. Smith, C1, Tuesday, ['10-11', '11-12']]
[INF304, Dr. Smith, C1, Friday, ['16-17', '17-18']]
[INF404, Dr. Smith, C3, Friday, ['10-11', '11-12']]
[INF504, Dr. Smith, C6, Friday, ['16-17', '17-18']]
[INF604, Dr. Smith, C5, Friday, ['16-17', '17-18']]
[INF

In [ ]:
def roulette_wheel_selection(population, number_of_parents):
    total_fitness = sum(ind.fitness for ind in population)
    selected_parents = []

    for _ in range(number_of_parents):
        pick = random.uniform(0, total_fitness)
        current = 0

        for ind in population:
            current += ind.fitness
            if current > pick:
                selected_parents.append(ind)
                break

    return selected_parents

In [ ]:
def multipoint_crossover(parent1, parent2):
    scheduled_lessons1 = parent1.scheduled_lessons
    scheduled_lessons2 = parent2.scheduled_lessons

    crossover_points = sorted(random.sample(range(len(scheduled_lessons1)), 2))
    point1, point2 = crossover_points

    offspring1_lessons = scheduled_lessons1[:point1] + scheduled_lessons2[point1:point2] + scheduled_lessons1[point2:]
    offspring2_lessons = scheduled_lessons2[:point1] + scheduled_lessons1[point1:point2] + scheduled_lessons2[point2:]

    offspring1 = Schedule(offspring1_lessons)
    offspring2 = Schedule(offspring2_lessons)

    return offspring1, offspring2

In [ ]:
def mutate_individual(scheduled_lesson):

    if random.random() < MUTATION_RATE:
        dice = random.randrange(1, 4)  
        if dice == 1:
            scheduled_lesson.classroom = random.choice(classrooms)
        elif dice == 2:
            scheduled_lesson.day = random.choice(days_of_week)
        elif dice == 3:
            possible_start_slots = [i for i in range(len(timeslots) - scheduled_lesson.lesson.duration + 1)]
            start_slot_index = random.choice(possible_start_slots)
            scheduled_lesson.start_timeslot = timeslots[start_slot_index:start_slot_index + scheduled_lesson.lesson.duration]

def mutation(offspring):
    for ind in offspring.scheduled_lessons:
        mutate_individual(ind)
    return offspring

In [ ]:
def elit_selection(population,offspring):
    combined_population = population + offspring

    new_population = sorted(combined_population, key=lambda ind: ind.fitness)[:POPULATION_SIZE]

    return new_population

In [ ]:
def evolutionary_algorithm():

    population = create_population()

    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        
        parents = roulette_wheel_selection(population, number_of_parents)
        
        offspring = []
        while len(offspring) < POPULATION_SIZE:
            parent1, parent2 = random.sample(parents, 2)
            child1, child2 = multipoint_crossover(parent1, parent2)
            offspring.append(mutation(child1))
            if len(offspring) < POPULATION_SIZE:
                offspring.append(mutation(child2))

        population = elit_selection(population,offspring)

        best_individual = min(population, key=lambda x: x.fitness)
        print("Best individual in this generation:")
        print(best_individual)
        
      
evolutionary_algorithm()